In [55]:
# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import re
import string

from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer,HashingVectorizer
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression

stopwords = stopwords.words('english')

In [56]:
train = pd.read_csv('no_stopwords_combined.csv')
test = pd.read_csv('testnostopwords.csv')

In [57]:
train.head()

,Unnamed: 0,title,text,subject,date,target
0,0,Ramaphosa's ANC election win lifts South Afric...,"['johannesburg', 'reuters', 'south', 'african'...",worldnews,"December 19, 2017",1
1,1,VIGILANTE PIRATES INTERCEDE Where Government F...,"['like', 'soldiers', 'oden', 'vigilante', 'gro...",Government News,"Apr 1, 2016",0
2,2,"SICK! DEMOCRAT ORGANIZER, Mayor DeBlasio Emplo...","['last', 'week', 'huma', 'abedin', 'husband', ...",left-news,"May 28, 2017",0
3,3,Dennis Rodman talks of skiing friendship with ...,"['edinburgh', 'reuters', 'us', 'basketball', '...",worldnews,"September 6, 2017",1
4,4,New Hampshire Cops Intent On Protecting Sadis...,"['lynchings', 'black', 'people', 'still', 'hap...",News,"September 10, 2017",0


In [58]:
test.head()

,article_title,text,location,target
0,Syria attack symptoms consistent with nerve ag...,"['wed', 'apr', 'syria', 'attack', 'symptoms', ...",idlib,0
1,Homs governor says U.S. attack caused deaths b...,"['fri', 'apr', 'homs', 'governor', 'says', 'us...",homs,0
2,Death toll from Aleppo bomb attack at least 112,"['sun', 'apr', 'death', 'toll', 'aleppo', 'bom...",aleppo,0
3,Aleppo bomb blast kills six Syrian state TV,"['wed', 'apr', 'aleppo', 'bomb', 'blast', 'kil...",aleppo,0
4,29 Syria Rebels Dead in Fighting for Key Alepp...,"['sun', 'jul', 'syria', 'rebels', 'dead', 'fig...",aleppo,0


In [66]:
'''
def clean_text(text):
    
    text = text.lower()
    text = re.sub('[\'\[\],]', '', text)

    return text

test['text'] = test['text'].apply(lambda x : clean_text(x))
train['text'] = train['text'].apply(lambda x : clean_text(x))
'''

In [67]:
test.head()

,article_title,text,location,target
0,Syria attack symptoms consistent with nerve ag...,wed apr syria attack symptoms consistent nerve...,idlib,0
1,Homs governor says U.S. attack caused deaths b...,fri apr homs governor says us attack caused de...,homs,0
2,Death toll from Aleppo bomb attack at least 112,sun apr death toll aleppo bomb attack least de...,aleppo,0
3,Aleppo bomb blast kills six Syrian state TV,wed apr aleppo bomb blast kills six syrian sta...,aleppo,0
4,29 Syria Rebels Dead in Fighting for Key Alepp...,sun jul syria rebels dead fighting key aleppo ...,aleppo,0


In [68]:
train.head()

,Unnamed: 0,title,text,subject,date,target
0,0,Ramaphosa's ANC election win lifts South Afric...,johannesburg reuters south african banking sto...,worldnews,"December 19, 2017",1
1,1,VIGILANTE PIRATES INTERCEDE Where Government F...,like soldiers oden vigilante group reported fe...,Government News,"Apr 1, 2016",0
2,2,"SICK! DEMOCRAT ORGANIZER, Mayor DeBlasio Emplo...",last week huma abedin husband anthony weiner w...,left-news,"May 28, 2017",0
3,3,Dennis Rodman talks of skiing friendship with ...,edinburgh reuters us basketball legend dennis ...,worldnews,"September 6, 2017",1
4,4,New Hampshire Cops Intent On Protecting Sadis...,lynchings black people still happen jim crow c...,News,"September 10, 2017",0


In [69]:
train_raw = train
test_raw = test

train_copy = train.copy(deep = True)
test_copy = test.copy(deep = True)

### Raw

In [71]:
count_vectorizer = CountVectorizer()

train_vectors = count_vectorizer.fit_transform(train_copy['text'])
test_vectors = count_vectorizer.transform(test_copy['text'])

train_vectors.shape, test_vectors.shape

((44898, 199319), (804, 199319))

### Min_DF and Max_DF parameter

In [72]:
count_vectorizer = CountVectorizer(min_df = 2,max_df = 0.8)

train_vectors = count_vectorizer.fit_transform(train_copy['text'])
test_vectors = count_vectorizer.transform(test_copy['text'])

train_vectors.shape, test_vectors.shape

((44898, 107650), (804, 107650))

### N-grams

In [74]:
#No text preprocessing req. REGEX and Stopwords are already taken care of

count_vectorizer = CountVectorizer(list(train_copy['text']), preprocessor = None,ngram_range = (1,2))

train_vectors = count_vectorizer.fit_transform(train_copy['text'])
test_vectors = count_vectorizer.transform(test_copy['text'])

train_vectors.shape, test_vectors.shape

((44898, 4048261), (804, 4048261))

In [77]:
list(count_vectorizer.vocabulary_)[:10]

['johannesburg',
 'reuters',
 'south',
 'african',
 'banking',
 'stocks',
 'rallied',
 'tuesday',
 'buoyed',
 'optimism']

In [79]:
# strict bigrams

count_vectorizer = CountVectorizer(list(train_copy['text']),
                                   preprocessor=None,
                                   ngram_range=(2,2))

train_vectors = count_vectorizer.fit_transform(train_copy['text'])
test_vectors = count_vectorizer.transform(test_copy['text'])

train_vectors.shape, test_vectors.shape

((44898, 3848942), (804, 3848942))

In [80]:
list(count_vectorizer.vocabulary_)[:10]

['johannesburg reuters',
 'reuters south',
 'south african',
 'african banking',
 'banking stocks',
 'stocks rallied',
 'rallied tuesday',
 'tuesday buoyed',
 'buoyed optimism',
 'optimism newly']

# New Trial Baseline

In [82]:
count_vectorizer = CountVectorizer(ngram_range=(1,2))

train_vectors = count_vectorizer.fit_transform(train_copy['text'])
test_vectors = count_vectorizer.transform(test_copy['text'])

X_train = train_vectors
y_train = train_copy['target']

In [83]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(C=1.0)
lr_scores = model_selection.cross_val_score(clf, X_train, y_train, cv=5, scoring="f1")

print(lr_scores)
print(lr_scores.sum()/5)

[0.99673203 0.99650105 0.99789818 0.99603082 0.99719953]
0.9968723214049511


### Testing on external dummy test data

In [84]:
clf.fit(train_vectors, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [90]:
from sklearn.metrics import f1_score, accuracy_score

y_test = test_copy['target']
preds = clf.predict(test_vectors)

print(f1_score(y_test,preds))

0.4915254237288135


# TF-IDF

In [92]:
# word level
tfidf = TfidfVectorizer(analyzer='word',
                        token_pattern=r'\w{1,}',
                        max_features=5000)

train_tfidf = tfidf.fit_transform(train_copy['text'])
test_tfidf = tfidf.transform(test_copy["text"])

In [93]:
#ngram level
tfidf = TfidfVectorizer(analyzer='word',
                        ngram_range=(2,3),
                        token_pattern=r'\w{1,}',
                        max_features=5000)

train_tfidf = tfidf.fit_transform(train_copy['text'])
test_tfidf = tfidf.transform(test_copy["text"])

In [94]:
# characters level
tfidf = TfidfVectorizer(analyzer='char',
                        ngram_range=(2,3),
                        token_pattern=r'\w{1,}',
                        max_features=5000)

train_tfidf = tfidf.fit_transform(train_copy['text'])
test_tfidf = tfidf.transform(test_copy["text"])

# Baseline with TF-IDF

In [95]:
train_tfidf = tfidf.fit_transform(train_copy['text'])
test_tfidf = tfidf.transform(test_copy["text"])

In [96]:
clf = LogisticRegression(C=1.0)
lr_scores = model_selection.cross_val_score(clf, 
                                            train_tfidf, 
                                            train_copy["target"], 
                                            cv=5, scoring="f1")

print(lr_scores)
print(lr_scores.sum()/5)

[0.97816696 0.98104872 0.98030533 0.97964406 0.97852583]
0.9795381776930858


In [97]:
clf.fit(train_tfidf, train_copy["target"])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [98]:
preds_tfidf = clf.predict(test_tfidf)

print(f1_score(y_test,preds_tfidf))

0.6178535606820461
